In [1]:
from hw1 import Composer
from midi2seq import process_midi_seq, seq2piano, random_piano, piano2seq, segment
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [3]:
expert_seq = process_midi_seq(maxlen=50,n=12000)
expert_seq.shape

(13149, 51)